In [ ]:
import matplotlib.widgets
from plantcv import plantcv as pcv
import videometer.hips as hips
import numpy as np
import argparse
import glob
import plantcv.plantcv as pcv
from skimage import img_as_ubyte

In [ ]:
# Set global debug behavior to None (default), "print" (to file), 
# or "plot" (Jupyter Notebooks or X11)
pcv.params.debug = "plot"
pcv.params.text_size = 5
pcv.params.text_thickness = 5

# Read in image
# Replace 'Path' with path to directory containing plates
# Replace 'X' in 'PlateX.raw' with plate number
img = pcv.readimage(filename="C:/Path/PlateX.raw", mode="envi")

In [ ]:
# Visualize image in 10 colorspaces as a pseudo RGB image
colorspace_img = pcv.visualize.colorspaces(rgb_img=img.pseudo_rgb)

In [ ]:
# Vizualize 19 bands of multispectral image
channels = [img_as_ubyte(img.array_data[:, :, i] / 255) for i in range(img.array_data.shape[2])]
tiles = pcv.visualize.tile(images=channels, ncol=4)

In [ ]:
# Convert pseudo rgb image to a gray HSV image, select channel 'h'
c_img = pcv.rgb2gray_hsv(rgb_img=img.pseudo_rgb, channel='h')

In [ ]:
# Threshold dark objects (seeds)
c_thresh = pcv.threshold.otsu(gray_img=c_img, object_type='dark')

In [ ]:
# Fill holes to remove spaces within seeds
c_fill = pcv.fill_holes(bin_img=c_thresh)

In [ ]:
# Define region of interest
roi = pcv.roi.rectangle(img=img.pseudo_rgb, x=277, y=100, h=2700, w=3750)

In [ ]:
# Fill objects less than 20000 pixels
a_fill_image = pcv.fill(bin_img=c_fill, size=20000)

In [ ]:
# Filter objects within ROI, keep seeds
flt_mask = pcv.roi.quick_filter(mask=a_fill_image, roi=roi)

In [ ]:
# Define grid of ROIs
rois = pcv.roi.auto_grid(mask=flt_mask, nrows=4, ncols=6, radius=250, img=img.pseudo_rgb)

In [ ]:
# Create labeled mask for data extraction
# Print number of ROIs identified
labeled_mask, num = pcv.create_labels(mask=flt_mask, rois=rois)
print(num)

In [ ]:
# Select image of wavelength 940nm
r940 = img_as_ubyte(img.array_data[: , :, -2] / 255)
pcv.plot_image(r940)

In [ ]:
# Apply seed mask to multispectral image
masked_r940 = pcv.apply_mask(img=r940, mask=flt_mask, mask_color="black")

In [ ]:
# Threshold out light objects (wings)
wing_mask = pcv.threshold.binary(gray_img=masked_r940, threshold=40, object_type="light")

In [ ]:
# Erode wing
wing_err = pcv.erode(gray_img=wing_mask, ksize=4, i=1)

In [ ]:
# Dilate wing
wing_mask_dil = pcv.dilate(gray_img=wing_err, ksize=4, i=2)

In [ ]:
# Fill small objects to eliminate noise
wing_mask_clean = pcv.fill(bin_img=wing_mask_dil, size=2500)

In [ ]:
# Subtract first wing mask from seed mask to leave kernels
kernel_mask = flt_mask - wing_mask_clean
pcv.plot_image(kernel_mask)

In [ ]:
# Erode kernel
kernel_err = pcv.erode(gray_img=kernel_mask, ksize=10, i=5)

In [ ]:
# Fill mask to eliminate noise
kernel_mask_clean = pcv.fill(bin_img=kernel_err, size=2900)

In [ ]:
# Dilate kernels
kernel_mask_dil = pcv.dilate(gray_img=kernel_mask_clean, ksize=8, i=4)

In [ ]:
# Fill holes in kernels
kernel_mask_fill = pcv.fill_holes(bin_img=kernel_mask_dil)

In [ ]:
# Create labeled mask for data extraction
kernel_labeled_mask, num = pcv.create_labels(mask=kernel_mask_fill, rois=rois)
print(num)

In [ ]:
# Subtract kernel mask from full mask
wing_mask_final = flt_mask - kernel_mask_fin
pcv.plot_image(wing_mask_fin)

In [ ]:
# Create labeled mask for data extraction
wing_labeled_mask, num = pcv.create_labels(mask=wing_mask, rois=rois)
print(num)

In [ ]:
# Whole seed data extraction
seed_shape_img = pcv.analyze.size(img=img.pseudo_rgb, labeled_mask=labeled_mask, n_labels=num, label="seed")
seed_hsv_img = pcv.analyze.color(rgb_img=img.pseudo_rgb, labeled_mask=labeled_mask, n_labels=num, label="seed")
seed_analysis_img = pcv.analyze.spectral_reflectance(hsi=img, labeled_mask=labeled_mask, n_labels=num, label='seed')

In [ ]:
# Wing data extraction
wing_shape_img = pcv.analyze.size(img=img.pseudo_rgb, labeled_mask=wing_labeled_mask, n_labels=num, label="wing")
wing_hsv_img = pcv.analyze.color(rgb_img=img.pseudo_rgb, labeled_mask=wing_labeled_mask, n_labels=num, label="wing")
wing_analysis_img = pcv.analyze.spectral_reflectance(hsi=img, labeled_mask=wing_labeled_mask, n_labels=num, label='wing')

In [ ]:
# Kernel data extraction
kernel_shape_img = pcv.analyze.size(img=img.pseudo_rgb, labeled_mask=kernel_labeled_mask, n_labels=num, label="kernel")
kernel_hsv_img = pcv.analyze.color(rgb_img=img.pseudo_rgb, labeled_mask=kernel_labeled_mask, n_labels=num, label="kernel")
kernel_analysis_img = pcv.analyze.spectral_reflectance(hsi=img, labeled_mask=kernel_labeled_mask, n_labels=num, label='kernel')

In [ ]:
modified_spectral = pcv.Spectral_data(array_data=analysis_img,
                           max_wavelength=list(img.wavelength_dict.keys())[-1],
                           min_wavelength=list(img.wavelength_dict.keys())[0],
                           max_value=float(np.amax(analysis_img)),
                           min_value=float(np.amin(analysis_img)),
                           d_type=modified_array_data.dtype,
                           wavelength_dict=img.wavelength_dict,
                           samples=modified_array_data.shape[1],
                           lines=modified_array_data.shape[0], interleave='bil',
                           wavelength_units=img.wavelength_units,
                           array_type="datacube",
                           pseudo_rgb=None,
                           filename=img.filename,
                           default_bands=None)

In [ ]:
args = options("F:/Landrigan/Fix_Thresholding", "F:/Landrigan/Fix_Thresholding/Results", True)
args